# MSABA-04: Pricing & Product Mix Optimization — Answer Key

In [ ]:
import pandas as pd
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpBinary, PULP_CBC_CMD

items = pd.read_csv("../data/items.csv")
import json, os
with open("../data/constraints.json","r") as f:
    C = json.load(f)

# Decision: buy quantity q_i and choose whether carry sku (y_i)
model = LpProblem("ProductMix", LpMaximize)
q = {r.sku: LpVariable(f"q_{r.sku}", lowBound=0, cat="Integer") for _, r in items.iterrows()}
y = {r.sku: LpVariable(f"y_{r.sku}", lowBound=0, upBound=1, cat="Binary") for _, r in items.iterrows()}

profit = lpSum([(r.price - r.unit_cost)*q[r.sku] for _, r in items.iterrows()])
model += profit

# Budget
model += lpSum([r.unit_cost * q[r.sku] for _, r in items.iterrows()]) <= C["budget"]
# Max SKUs
model += lpSum([y[r.sku] for _, r in items.iterrows()]) <= C["max_skus"]
# Demand caps and linking q <= demand_cap*y
for _, r in items.iterrows():
    model += q[r.sku] <= r.demand_cap * y[r.sku]

model.solve(PULP_CBC_CMD(msg=False))

sol = []
for _, r in items.iterrows():
    sol.append({"sku": r.sku, "qty": q[r.sku].value(), "selected": int(y[r.sku].value())})
out = pd.DataFrame(sol).sort_values("selected", ascending=False)
print(out.head(15))
print("Objective (profit):", round(model.objective.value(),2))
